# Light curve

In this notebook we'll assemble the light curve of a star, calibrated against other stars of the same field, with the purpose of uncovering an exoplanet transit.

As usual, you can download this page as a {download}`jupyter notebook <./lightcurve.ipynb>` file.

In [ ]:
import dataredconfig
import pathlib

import numpy as np
import astropy
import astropy.table
import astropy.visualization
from astropy import units as u

import datetime

%matplotlib widget
import matplotlib
from matplotlib import pyplot as plt

import ccdproc

In [ ]:
photometry_dir = dataredconfig.work_dir / "PHOTOMETRY"

object_to_process = "HD92670"

catalog_filepaths = sorted(list(photometry_dir.glob('*.fits')))
catalogs = []

for catalog_filepath in catalog_filepaths:
    
    catalog = astropy.table.Table.read(catalog_filepath)

    # We select the photometric catalogs of our object:  
    if "OBJECT" in catalog.meta:
        if catalog.meta["OBJECT"] == object_to_process:
            print(f"{catalog_filepath} : {catalog.meta}")
            catalogs.append(catalog)


In [ ]:
# We combine these catalogs into a single table, in "depth": columns will be 2D
catalog = astropy.table.dstack(catalogs, join_type="exact", metadata_conflicts="silent")

# We also produce a list of datetime objects, from the FITS headers:
date_strings = [c.meta["DATE-OBS"] for c in catalogs]
dates = [datetime.datetime.fromisoformat(date) for date in date_strings]

# And while we are at it, same for the airmass:
airmasses = [c.meta["AIRMASS"] for c in catalogs]

# We read the reference catalog, as this one contains the position of each star
ref_catalog = astropy.table.Table.read(photometry_dir / f"ref_catalog_{object_to_process}.fits")
assert len(ref_catalog) == len(catalog) # Just a check that these are indeed of same length

# We copy the positions from the reference catalog over to our combined catalog:
catalog["sky_centroid_win"] = ref_catalog["sky_centroid_win"]

# No need for all the rest...
#catalog = astropy.table.hstack([ref_catalog, catalog],
#        join_type="exact", metadata_conflicts="silent")

print(f"Number of epochs: {len(catalogs)}")
catalog.colnames


We display an image of the field, overplotting the indices of catalog sources

In [ ]:
# We load one of the images, it does not have to be a specific one.
light_prered_dir = dataredconfig.work_dir / "LIGHT_PRERED"
science_files = ccdproc.ImageFileCollection(light_prered_dir, keywords=dataredconfig.ifc_header_keywords)
science_files = science_files.filter(object=object_to_process)
image_path = science_files.files[0]
image = ccdproc.CCDData.read(image_path, unit="adu")
image.data -= np.median(image.data) # Quick sky subtraction

# And now create the figure
plt.figure(figsize=(10, 6))
ax = plt.subplot(projection=image.wcs)
ax.imshow(image.data, origin='lower', cmap='Greys_r', interpolation='nearest',
    norm=astropy.visualization.simple_norm(image.data, stretch="sqrt", min_cut=-20, max_cut=500))
ax.scatter(
    catalog["sky_centroid_win"].ra.degree,
    catalog["sky_centroid_win"].dec.degree,
    transform=ax.get_transform('world'),
    s=50, # The size of these markers is not related to any measurement apertures!
    edgecolor='red', facecolor='none'
    )
for line in catalog:
    ax.text(
        x=line["sky_centroid_win"].ra.degree,
        y=line["sky_centroid_win"].dec.degree,
        s=str(line.index),
        transform=ax.get_transform('world'),
        color="cyan"
        )
ax.grid(color='white', ls='solid')
ax.coords[0].set_axislabel('RA')
ax.coords[1].set_axislabel('Dec')
#ax.coords[0].set_ticks(spacing=5.*u.arcmin)
#ax.coords[1].set_ticks(spacing=5.*u.arcmin)
plt.tight_layout()
plt.show()



In [ ]:
target_index = 63
target_mag = 11.8
target_mag_first_n_images = 20
transit_depth_ppt = 14.7
n_bright_stars_calib = 30
n_bright_stars_show = 5


ingress_datetime = datetime.datetime.fromisoformat("2024-04-21T22:32") # Both in UTC
egress_datetime = datetime.datetime.fromisoformat("2024-04-22T00:18")
title_str = "WASP-104 b (Moon 96% @31°)"

catalog["instr_mag"] = -2.5 * np.log10(catalog["sum_10"].value) # a "2D" column (index, date)

zero_point = target_mag - np.nanmedian(catalog[target_index]["instr_mag"])
print(f"Zero-point: {zero_point}")
catalog["mag"] = catalog["instr_mag"] + zero_point

catalog["medianmag"] = np.nanmedian(catalog["mag"], axis=1) # one value per source

catalog["flux_sort_indices"] = np.argsort(catalog["medianmag"])
indices_calib = catalog["flux_sort_indices"][0:n_bright_stars_calib]
indices_show = catalog["flux_sort_indices"][1:n_bright_stars_show]

catalog["delta_mag"] = catalog["mag"] - np.expand_dims(catalog["medianmag"], axis=1) # 2D - 1D -> 2D (index, date)


same_calib_for_all = np.nanmedian(catalog["delta_mag"][indices_calib], axis=0)
# Normalize this:
same_calib_for_all -= np.mean(same_calib_for_all)
# And make it a colum of the catalog
catalog["calib"] = np.tile(same_calib_for_all, (len(catalog), 1))

transit_depth_mag = 2.5*np.log10(1.0/(1.0-transit_depth_ppt/1000.0))
                                 
plt.figure()
ax = plt.subplot()
ax.axvline(ingress_datetime, lw=1)
ax.axvline(egress_datetime, lw=1)
ax.axhline(target_mag, lw=1, ls="--")
ax.axhline(target_mag + transit_depth_mag, lw=1, ls=":")

# Plot some field stars in grey:
for (i, index) in enumerate(indices_show):
    ax.plot(
    dates, catalog["mag"][index] - catalog["calib"][target_index] - (catalog["medianmag"][index] - catalog["medianmag"][target_index]) - 0.015*(i+1), 
    color="grey", lw=1
    )

# Plot the target in red:
ax.plot(
    dates, catalog["mag"][target_index] - catalog["calib"][target_index], 
    color="red"
    )

ax.invert_yaxis()

ax2 = ax.twinx()
color="purple"
ax2.set_ylabel('Airmass', color=color)
ax2.plot(dates, airmasses, color=color, lw=1, ls="-.")
ax2.tick_params(axis='y', labelcolor=color)


ax.xaxis.set_major_formatter(matplotlib.dates.ConciseDateFormatter(ax.xaxis.get_major_locator()))
#plt.colorbar(label=f"Separation to target center in {cat['separation'].unit}")
#plt.xlabel("Date")
ax.set_ylabel("i Magnitude", color="red")
plt.gcf().autofmt_xdate()
plt.title(title_str)
plt.show()
